#1. Libraries

In [40]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
import numpy as np
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 8))
import seaborn as sns
sns.set(color_codes=True)
import random
import pickle

from datetime import datetime

!pip install pytictoc
from pytictoc import TicToc

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import  roc_curve, auc, recall_score, precision_score,f1_score, confusion_matrix, mean_squared_error
from sklearn.model_selection import GridSearchCV, ParameterGrid
from sklearn.calibration import CalibratedClassifierCV, calibration_curve

!pip install catboost
from catboost import CatBoostClassifier

#2. Data

In [41]:
path = "/content/drive/MyDrive/ColabNotebooks/coverwallet_codetest/assets/"
output = "/content/drive/MyDrive/ColabNotebooks/coverwallet_codetest/output/"
model_alg = "randomforest_cal" #do not use catbost here

robust_scaler = pickle.load(open(path + "robustscaler_pkl", 'rb'))
model_trained = pickle.load(open(path + model_alg + "_pkl", 'rb'))

In [42]:
#Read received csv
f_accounts_test = '/content/drive/MyDrive/ColabNotebooks/coverwallet_codetest/data/accounts_test.csv'
f_quotes_test = '/content/drive/MyDrive/ColabNotebooks/coverwallet_codetest/data/quotes_test.csv'

df_accounts_test = pd.read_csv(f_accounts_test)
df_quotes_test = pd.read_csv(f_quotes_test)

In [43]:
#Datasets dimensions
num_rows_acc, num_cols_acc = df_accounts_test.shape
num_rows_quo, num_cols_quo = df_quotes_test.shape
print("The accounts dataset has {0} rows and {1} columns".format(num_rows_acc,num_cols_acc))
print("The quotes dataset has {0} rows and {1} columns".format(num_rows_quo,num_cols_quo))

The accounts dataset has 2448 rows and 9 columns
The quotes dataset has 5005 rows and 4 columns


In [44]:
df_accounts_test.dtypes

account_uuid           object
state                  object
industry               object
subindustry            object
year_established      float64
annual_revenue        float64
total_payroll         float64
business_structure     object
num_employees         float64
dtype: object

In [45]:
df_quotes_test.dtypes

account_uuid     object
product          object
premium         float64
carrier_id        int64
dtype: object

In [46]:
df_quotes_test["carrier_id"] = df_quotes_test["carrier_id"].astype(str)

#3. Duplicates

In [47]:
#Unnecessarily duplicated rows
if df_accounts_test.duplicated().any():
    print("Duplicated full rows in accounts dataset")
if df_quotes_test.duplicated().any():
    print("Duplicated full rows in quotes dataset")  
else: print("Everything is ok")

Duplicated full rows in quotes dataset


In [48]:
quotes_duplicates = df_quotes_test[df_quotes_test.duplicated()]
print(len(quotes_duplicates))

359


In [49]:
df_quotes_test = df_quotes_test[~df_quotes_test.duplicated()]

In [50]:
len(pd.unique(df_quotes_test["account_uuid"]))

2448

#4. Null values

In [51]:
df_quotes_test.isnull().sum()

account_uuid    0
product         0
premium         0
carrier_id      0
dtype: int64

In [52]:
values = {"industry": "blank", "subindustry": "blank"}
df_accounts_test.fillna(value=values, inplace=True)

In [53]:
df_accounts_test.isnull().sum()

account_uuid           0
state                  3
industry               0
subindustry            0
year_established       6
annual_revenue         5
total_payroll         21
business_structure     0
num_employees         12
dtype: int64

In [54]:
values = {"state": "FL", "year_established": 2017, "annual_revenue": 217706.9, "total_payroll": 43469.87, "num_employees": 8}
df_accounts_test.fillna(value=values, inplace=True)

#5. Merge datasets

In [55]:
df_quotes_test = df_quotes_test.rename({"account_uuid": "company_id"}, axis="columns")

In [56]:
df_accounts_full_test = df_accounts_test.merge(df_quotes_test, how="inner", left_on=["account_uuid"],right_on=["company_id"])
num_rows_quo, num_cols_quo = df_accounts_full_test.shape
print("The complete dataset has {0} rows and {1} columns".format(num_rows_quo,num_cols_quo))

The complete dataset has 4646 rows and 13 columns


In [57]:
df_accounts_full_test.drop(["company_id", "subindustry"], axis=1, inplace=True)

# 6. Creating necessary variables

GROUPING RESIDUAL VALUES

In [58]:
#Grouping residual values
major_bu_structures = ["Limited Liability Company", "Individual", "Corporation"]
major_products = ["General_Liability", "Business_Owners_Policy_BOP", "Workers_Compensation", "CW_Professional_Liability", "Commercial_Auto", "CW_Errors_Omissions_E_O", "CW_Umbrella_Policy", "Package"]
major_carriers = ["39", "31", "60","30","29","9","53","21","73","22","40","72"]
major_states = ["FL","CA","NY","TX","GA","NJ","VA","PA","NC","SC","IL","LA","MA","OH","MI","CO","MD","AL","TN","MO","AZ","WA","WI","IN","NV","OK","KY","MS","OR"]
major_industries = ["Contractors", "Retail Trade", "Other Services", "Professional, Scientific and Technical Services", "Administrative Services and Building Maintenance", "Food and Accommodation", "Consultants", "Technology, Media and Telecommunications", "Manufacturing", "Healthcare", "Wholesale Trade", "Transportation and Warehousing", "Real Estate", "Sports, Arts, Entertainment, and Recreation", "Construction", "Finance and Insurance", "Education" ]
#blank industry is left out and back into residual group

def grouping_residuals(var_value: str, major_values: list) -> str:
  if var_value in major_values:
    return var_value
  else:
    return "residuals"

df_accounts_full_test['product_res'] = df_accounts_full_test['product'].apply(lambda x: grouping_residuals(x, major_products))
df_accounts_full_test['carrier_id_res'] = df_accounts_full_test['carrier_id'].apply(lambda x: grouping_residuals(x, major_carriers))
df_accounts_full_test['state_res'] = df_accounts_full_test['state'].apply(lambda x: grouping_residuals(x, major_states))
df_accounts_full_test['industry_res'] = df_accounts_full_test['industry'].apply(lambda x: grouping_residuals(x, major_industries))
df_accounts_full_test['business_structure_res'] = df_accounts_full_test['business_structure'].apply(lambda x: grouping_residuals(x, major_bu_structures))

In [59]:
df_accounts_full_test.drop(["product", "carrier_id", "state", "industry", "business_structure"], axis=1, inplace=True)

DUMMIES

In [60]:
def string_to_nomenclature(text):
    return text.strip().replace(',','_').replace('/','_').replace('.','_').replace(' ','_').replace('-','_').replace('(','_').replace(')','_').lower()

In [61]:
if model_alg != "catboost":
  name_vars_object = [name for name,tipo in df_accounts_full_test.dtypes.iteritems() if 'object' in str(tipo) and name not in ["account_uuid"]]
  df_accounts_full_test[name_vars_object] = df_accounts_full_test[name_vars_object].applymap(lambda x: string_to_nomenclature(x))

  #Building dummies in a separate dataset
  df_accounts_full_test_dummies = pd.get_dummies(df_accounts_full_test,columns=name_vars_object)
else:
  df_accounts_full_test_dummies = df_accounts_full_test

CALCULATED VARIABLES

In [62]:
df_accounts_full_test_dummies["year_established"].astype(str).value_counts()

2017.0    1690
2018.0     921
2016.0     367
2015.0     294
2014.0     234
2012.0     143
2013.0     134
2008.0      99
2010.0      87
2009.0      73
2007.0      62
2011.0      54
2005.0      48
2003.0      45
2006.0      45
2004.0      44
2002.0      31
2001.0      30
2000.0      30
1993.0      17
1999.0      16
1998.0      16
1992.0      13
1996.0      12
1989.0      12
1986.0      11
1982.0      11
1990.0      11
1987.0      11
1994.0      11
1995.0       9
1985.0       9
1997.0       9
1988.0       8
1991.0       6
1984.0       6
1977.0       5
1972.0       3
1983.0       3
1981.0       2
1978.0       2
1943.0       2
1975.0       2
1952.0       1
1961.0       1
1979.0       1
1966.0       1
1973.0       1
1949.0       1
1885.0       1
1967.0       1
Name: year_established, dtype: int64

In [63]:
df_accounts_full_test_dummies["company_age"] = pd.datetime.now().year - df_accounts_full_test_dummies["year_established"]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  """Entry point for launching an IPython kernel.


In [64]:
if model_alg != "catboost":
  df_accounts_full_test_dummies.drop(["year_established", "total_payroll", "carrier_id_res_53", "business_structure_res_individual"], axis=1, inplace=True)
else:
  df_accounts_full_test_dummies.drop(["year_established", "total_payroll"], axis=1, inplace=True)


#7. Applying Model

In [65]:
if model_alg != "catboost":
  model_features = ['premium', 'annual_revenue', 'company_age', 'num_employees', 'carrier_id_res_60', 'business_structure_res_limited_liability_company', 'product_res_business_owners_policy_bop', 'carrier_id_res_39', 'carrier_id_res_31', 'industry_res_other_services', 'business_structure_res_corporation', 'state_res_fl', 'industry_res_retail_trade', 'product_res_general_liability', 'state_res_ca', 'industry_res_contractors', 'state_res_residuals', 'industry_res_food_and_accommodation', 'industry_res_professional__scientific_and_technical_services', 'state_res_ny', 'state_res_tx', 'state_res_ga', 'industry_res_administrative_services_and_building_maintenance', 'industry_res_consultants', 'business_structure_res_residuals', 'state_res_va', 'industry_res_manufacturing', 'state_res_nj', 'state_res_nc', 'state_res_il', 'state_res_sc', 'carrier_id_res_residuals', 'carrier_id_res_30', 'carrier_id_res_21', 'carrier_id_res_9', 'state_res_la', 'state_res_pa', 'state_res_mi', 'industry_res_technology__media_and_telecommunications', 'carrier_id_res_29', 'state_res_oh', 'state_res_tn', 'industry_res_wholesale_trade', 'industry_res_healthcare', 'product_res_residuals', 'state_res_md', 'state_res_co', 'state_res_al', 'state_res_ma', 'state_res_mo', 'state_res_az', 'state_res_wi', 'product_res_workers_compensation', 'industry_res_education', 'industry_res_residuals', 'product_res_cw_professional_liability', 'product_res_commercial_auto', 'state_res_nv']
else:
  model_features = ["premium","annual_revenue","num_employees","product_res","carrier_id_res","state_res" ,"industry_res" ,"business_structure_res","company_age"]

df_results = df_accounts_full_test_dummies[["account_uuid"] + model_features]

In [66]:
features_to_standardize = ["premium","annual_revenue","company_age","num_employees"]
df_results["premium_orig"] = df_results["premium"]
df_results[features_to_standardize] = robust_scaler.transform(df_results[features_to_standardize])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


In [67]:
results = df_results[["account_uuid", "premium_orig"]]
results["prob"] = [pair[1] for pair in model_trained.predict_proba(df_results.drop(["account_uuid", "premium_orig"], axis=1))] #right side of the tuple has the probability of conversion = 1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


#8. Account Value calculation

In [68]:
results["exp_accvalue"] = results["prob"] * results["premium_orig"]
results_rmse = pd.DataFrame(results.groupby("account_uuid").agg({"exp_accvalue": sum})).reset_index()
results_rmse

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,account_uuid,exp_accvalue
0,0007250d-1fa22-3ea7-f8d9a-2ab585d136,1713.692950
1,000f36a0-70dec-6b10-04a24-d07bcbe4ed,1144.959514
2,00143506-a4ba9-6d4d-558b4-28e36e9ee4,6292.454523
3,001d5526-eedf-49c9-904a-c4297022c4c5,530.201791
4,002d4d12-184f-4e8c-8c79-851ec406c568,337.913040
...,...,...
2443,ff4b624e-906e6-24e3-a128b-ee25dab4ce,513.776764
2444,ff5b8d3e-5842-4495-b7bc-8c20bdac4bf8,4400.532369
2445,ff7ba7a6-2d9a-4e26-8344-d2254e2cd747,2329.471847
2446,ffd84807-18c22-ea59-71f3a-133f375af5,1385.847148


In [69]:
now = datetime.now().strftime("%Y%m%d_%H%M")
results_rmse.to_csv(output + now + '_submission.csv', index=False)  